In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(llm=llm,
                                  max_token_limit=200,
                                  return_messages=True,
                                  memory_key="chat_history")



def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]




prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
    "chat_history": load_memory,
}  | prompt | llm

chain
def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {'input': question},
        {'output': result},
    )
    print(result)
    
invoke_chain('Is Aaronson guilty?')


{
  documents: VectorStoreRetriever(tags=['FAISS', 'CacheBackedEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x299410810>),
  question: RunnablePassthrough(),
  chat_history: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="\n            Given the following extracted parts of a long document and a question, create a final answer. \n            If you don't know the answer, just say that you don't know. Don't try to make up an answer.\n            ------\n            {context}\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromp

In [ ]:
chain.invoke("Is Aaronson guilty?")

KeyError: 'context'